## 환경 설정

In [109]:
# local root
rootdir = 'E:\\Retropie'

# dirs to sync
games = [
    'arcade',
    'megadrive',
    'nes',
    'snes',
    'msx',
    'pcengine',
    'segacd',
    'fds',
    'gba',
    'mastersystem',
]

# remote rom direcgtory
romdir = '/home/pi/roms/'

In [110]:
import os
# rootdir = os.getcwd()

## 레트로 파이 SSH 접속하기

In [111]:
from paramiko import SSHClient
from scp import SCPClient

ssh = SSHClient()
ssh.load_system_host_keys()

C:\Users\k5020\Miniconda3\envs\py37\lib\site-packages\paramiko\ecdsakey.py:164: CryptographyDeprecationWarning: Support for unsafe construction of public numbers from encoded data will be removed in a future version. Please use EllipticCurvePublicKey.from_encoded_point
  self.ecdsa_curve.curve_class(), pointinfo


In [112]:
ssh.connect(username='pi',password='raspberry', hostname='retropie')
scp = SCPClient(ssh.get_transport())

C:\Users\k5020\Miniconda3\envs\py37\lib\site-packages\paramiko\kex_ecdh_nist.py:39: CryptographyDeprecationWarning: encode_point has been deprecated on EllipticCurvePublicNumbers and will be removed in a future version. Please use EllipticCurvePublicKey.public_bytes to obtain both compressed and uncompressed point encoding.
  m.add_string(self.Q_C.public_numbers().encode_point())
C:\Users\k5020\Miniconda3\envs\py37\lib\site-packages\paramiko\kex_ecdh_nist.py:96: CryptographyDeprecationWarning: Support for unsafe construction of public numbers from encoded data will be removed in a future version. Please use EllipticCurvePublicKey.from_encoded_point
  self.curve, Q_S_bytes
C:\Users\k5020\Miniconda3\envs\py37\lib\site-packages\paramiko\kex_ecdh_nist.py:111: CryptographyDeprecationWarning: encode_point has been deprecated on EllipticCurvePublicNumbers and will be removed in a future version. Please use EllipticCurvePublicKey.public_bytes to obtain both compressed and uncompressed point en

## 필요 함수 정의

In [113]:
def getRemotes(gamedir):
    stdin, stdout, stderr = ssh.exec_command('ls ~/roms/' + gamedir)
    remote_dict = {}
    for fname in stdout:
        remote_dict[fname[:-1]] = 1
    return remote_dict

In [114]:
# for game in games:
#     rdict = getRemotes(game)
#     print(game)
#     print(len(game))
#     print(list(rdict.keys())[:5])

In [115]:
def getLocals(gamedir):
    os.chdir(rootdir)
    os.chdir(gamedir)
    locals = os.listdir()
    local_dict = {}
    for f in locals:
        local_dict[f] = 1
    return local_dict

In [116]:
def copyFiles(rdict, ldict, gamedir):
    count = 0
    for r in rdict.keys():
        if r not in ldict or r == 'gamelist.xml':
            print("Backup file: {}".format(r))
            scp.get("{}/{}/{}".format(romdir, gamedir, r), recursive=True)
            count += 1
#         else:
#             print("Ok " + r)
    return count

In [117]:
def rbackup(gamedir):
    print("Start backup " + gamedir)
    rdict = getRemotes(gamedir)
    ldict = getLocals(gamedir)
    count = copyFiles(rdict, ldict, gamedir)    
    print("End backup {} : {}\n================\n".format(gamedir, count))
    os.chdir(rootdir)

In [118]:
def backupSnap(gamedir):
    snapdir = gamedir + '/snap'
    print("Start backup snap" + gamedir)
    rdict = getRemotes(snapdir)
    ldict = getLocals(snapdir)
    count = copyFiles(rdict, ldict, snapdir)    
    print("End backup {} : {}\n================\n".format(gamedir, count))
    os.chdir(rootdir)    

In [119]:
for game in games:
    rbackup(game)
    backupSnap(game)
print("All done!")

Start backup arcade
Backup file: gamelist.xml
End backup arcade : 1

Start backup snaparcade
End backup arcade : 0

Start backup megadrive
Backup file: gamelist.xml
End backup megadrive : 1

Start backup snapmegadrive
End backup megadrive : 0

Start backup nes
Backup file: gamelist.xml
End backup nes : 1

Start backup snapnes
End backup nes : 0

Start backup snes
Backup file: gamelist.xml
End backup snes : 1

Start backup snapsnes
End backup snes : 0

Start backup msx
Backup file: gamelist.xml
End backup msx : 1

Start backup snapmsx
End backup msx : 0

Start backup pcengine
Backup file: gamelist.xml
End backup pcengine : 1

Start backup snappcengine
End backup pcengine : 0

Start backup segacd
Backup file: gamelist.xml
End backup segacd : 1

Start backup snapsegacd
End backup segacd : 0

Start backup fds
Backup file: gamelist.xml
End backup fds : 1

Start backup snapfds
End backup fds : 0

Start backup gba
Backup file: gamelist.xml
End backup gba : 1

Start backup snapgba
End backup g

## clean local directory 

아래의 코드를 싫행하면 로컬의 파일을 제거하므로 주의할 것

In [104]:
def removeFiles(rdict, ldict, gamedir):
    count = 0
    for lfile in ldict.keys():
        if lfile not in rdict:
            print(lfile)
            os.remove(lfile)
            count += 1
#         else:
#             print(lfile)
    return count

In [121]:
def localClear(gamedir):
    print("Start clear local " + gamedir)
    rdict = getRemotes(gamedir)
    ldict = getLocals(gamedir)
    count = removeFiles(rdict, ldict, gamedir)    
    print("End clean {} : {}\n================\n".format(gamedir, count))
    os.chdir(rootdir)
    

In [125]:
def localSnapClear(gamedir):
    snapdir = gamedir + '/snap'
    print("Start clear local snap " + gamedir)
    rdict = getRemotes(snapdir)
    ldict = getLocals(snapdir)
    count = removeFiles(rdict, ldict, snapdir)    
    print("End clean {} : {}\n================\n".format(gamedir, count))
    os.chdir(rootdir)
    

In [128]:
localSnapClear('arcade')

Start clear local snap arcade
3wonders-image - 복사본.png
End clean arcade : 1



In [129]:
for game in games:
    localClear(game)
    localSnapClear(game)
print("All done!")

Start clear local arcade
End clean arcade : 0

Start clear local snap arcade
End clean arcade : 0

Start clear local megadrive
End clean megadrive : 0

Start clear local snap megadrive
End clean megadrive : 0

Start clear local nes
End clean nes : 0

Start clear local snap nes
End clean nes : 0

Start clear local snes
End clean snes : 0

Start clear local snap snes
End clean snes : 0

Start clear local msx
End clean msx : 0

Start clear local snap msx
End clean msx : 0

Start clear local pcengine
End clean pcengine : 0

Start clear local snap pcengine
End clean pcengine : 0

Start clear local segacd
End clean segacd : 0

Start clear local snap segacd
End clean segacd : 0

Start clear local fds
End clean fds : 0

Start clear local snap fds
End clean fds : 0

Start clear local gba
End clean gba : 0

Start clear local snap gba
End clean gba : 0

Start clear local mastersystem
End clean mastersystem : 0

Start clear local snap mastersystem
End clean mastersystem : 0

All done!


In [92]:
# scp.put('test.txt', 'test2.txt')
# scp.get(rdir + 'test2.txt')

# # Uploading the 'test' directory with its content in the
# # '/home/user/dump' remote directory
# scp.put('test', recursive=True, remote_path='/home/user/dump')

# scp.close()